In [1]:
import re
import time
import requests
import numpy as np

from bs4 import BeautifulSoup
from collections import Counter
from selenium import webdriver

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd

from selenium.webdriver.common.by import By


In [2]:
url = 'https://www.usajobs.gov/Search/Results?jt=Data%20Analyst'

In [3]:
## launch headless Edge instance
options = webdriver.EdgeOptions()
options.add_argument('--headless')

driver = webdriver.Edge(options = options)
driver.get(url)

## grab html from page
page = driver.page_source.encode('utf-8')
soup = BeautifulSoup(page, 'html.parser')

## find total number of jobs found
job_count_ele = soup.find(id = 'page-info')
print(job_count_ele.prettify())


<h4 class="usajobs-search-controls__results-count" id="page-info">
 Viewing 1 – 25 of 1538 jobs
</h4>



In [4]:
job_count = int(re.search(r'[0-9]+(?= jobs)', str(job_count_ele)).group())
print(job_count)

1538


In [ ]:
## initiate variables for loop
df = pd.DataFrame(columns = ['job_title', 'job_url','job_summary', 'job_salary', 'job_location', 'job_quals', 'job_duties'])
words = []
counter = 0
start = time.time()
i = 0
for ii in range(10): 
    
    print("Starting")
    print(ii)
    page = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(page, 'html.parser')
    job_rows = soup.find_all('a', class_ = 'usajobs-search-result--core__title') 
    print("Number of jobs")
    print(len(job_rows))

    ## loop through rows, grabbing unique job posting url from each  
    for iii in range(len(job_rows)): # 
        
        ## grab url for unique job post
        job_link = re.search(r'(?<=href=\").*(?=\" id)', str(job_rows[iii])).group()
        print(job_link)
        j_url = 'https://www.usajobs.gov/'
        job_url = str(j_url + 
                      re.search(r'(?<=href=\").*(?=\" id)', 
                                str(job_rows[iii])).group()
                     )

        ## capture html from post page
        job_page = requests.get(job_url)
        job_soup = BeautifulSoup(job_page.content, 'html.parser')
        
        # grab job title
        job_title = job_soup.find('h1', class_ = 'usajobs-joa-banner__title')
        job_title = re.sub(r'\<.*\>', '', job_title.prettify())

        # grab job summary
        job_summary = job_soup.find('div', id = 'summary')
        job_summary = re.sub(r'\<.*\>', '', job_summary.prettify())
        
        # grab job description
        job_salary = job_soup.find('p', class_ = 'usajobs-joa-summary__salary salary-text-normal')
        job_salary = re.sub(r'\<.*\>', '', job_salary.prettify())
        
        # grab job location
        job_location = job_soup.find('div', class_ = "usajobs-joa-locations__body")
        job_location = re.sub(r'\<.*\>', '', job_location.prettify())
        
        # grab job qualifications
        job_quals = job_soup.find('div', id = "qualifications")
        job_quals = re.sub(r'\<.*\>', '', job_quals.prettify())
        
        job_duties = job_soup.find('div', id = "duties")
        job_duties = re.sub(r'\<.*\>', '', job_duties.prettify())
        
        
        ## move all variables to dataframe
        df.loc[i, 'job_title'] = job_title
        df.loc[i,'job_url'] = job_url
        df.loc[i,'job_summary'] = job_summary
        df.loc[i,'job_salary'] = job_salary
        df.loc[i,'job_location'] = job_location
        df.loc[i,'job_quals'] = job_quals
        df.loc[i,'job_duties'] = job_duties
        
        time.sleep(4)
        
        i = i + 1
        
    df2 = df
    df2.job_title = df2.job_title.str.replace('[^a-zA-Z0-9]', ' ')
    df2.job_summary = df2.job_summary.str.replace('[^a-zA-Z0-9]', ' ')
    df2.job_location = df2.job_location.str.replace('[^a-zA-Z0-9]', ' ')
    df2.job_quals = df2.job_quals.str.replace('[^a-zA-Z0-9]', ' ')
    df2.job_salary = df2.job_salary.str.replace('[^0-9]','-')
    df2.job_duties = df2.job_duties.str.replace('[^a-zA-Z0-9]',' ')
    df2.to_csv("data_analyst.csv")
        
    button = driver.find_element(By.CLASS_NAME, 'usajobs-search-pagination__next-page')
    driver.execute_script('arguments[0].click()', button)
    time.sleep(2)

        

        
        

Starting
0
Number of jobs
25
/job/705283300
/job/709658500
/job/709166200
/job/709017400
/job/710689400
/job/711718200
/job/711718700
/job/712013300
/job/711421500
/job/711218900
/job/709381700
/job/709382200
/job/709994900
/job/710921700
/job/709248300
/job/709250300
/job/709827100
/job/709828500
/job/711682100
/job/652648400
/job/707297000
/job/708597000
/job/708842100
/job/708949100
/job/708949600


C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:71: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_title = df2.job_title.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_summary = df2.job_summary.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:73: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_location = df2.job_location.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_quals = df2.job_quals.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:75: FutureWarning:

Starting
1
Number of jobs
25
/job/708958100
/job/709571700
/job/709580700
/job/710110600
/job/710743700
/job/710744000
/job/710897600
/job/711153200
/job/711306400
/job/711306600
/job/711338200
/job/711343400
/job/711368100
/job/711702800
/job/711703400
/job/709670500
/job/706514600
/job/709213200
/job/709666100
/job/711116800
/job/711708800
/job/711710100
/job/711385900
/job/711636400
/job/711636500


C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:71: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_title = df2.job_title.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_summary = df2.job_summary.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:73: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_location = df2.job_location.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_quals = df2.job_quals.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:75: FutureWarning:

Starting
2
Number of jobs
25
/job/711462600
/job/711328500
/job/711548000
/job/711774600
/job/703548200
/job/709063100
/job/710542900
/job/711456300
/job/709606500
/job/709608500
/job/709708600
/job/709708700
/job/710310800
/job/710983900
/job/710585000
/job/709955000
/job/710129800
/job/710328000
/job/710700500
/job/711427100
/job/712053500
/job/687971500
/job/688790200
/job/690711800
/job/708044000


...

Starting
6
Number of jobs
25
/job/706505100
/job/707724300
/job/707808100
/job/708050000
/job/708067700
/job/708465500
/job/708484800
/job/708626300
/job/708628100
/job/708666000
/job/708837400
/job/708841500
/job/708874400
/job/709021400
/job/709065700
/job/709076200
/job/709076500
/job/709123000
/job/709165200
/job/709210300
/job/709245100
/job/709263600
/job/709265200
/job/709281800
/job/709379100


C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:71: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_title = df2.job_title.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_summary = df2.job_summary.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:73: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_location = df2.job_location.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_quals = df2.job_quals.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:75: FutureWarning:

Starting
7
Number of jobs
25
/job/709379200
/job/709384100
/job/709430700
/job/709445400
/job/709445500
/job/709567500
/job/709585000
/job/709587200
/job/709587700
/job/709615300
/job/709625400
/job/709636300
/job/709637200
/job/709688100
/job/709688700
/job/709700400
/job/709701400
/job/709726900
/job/709811500
/job/709817400
/job/709839600
/job/709905100
/job/709937000
/job/709946000
/job/710062800


C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:71: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_title = df2.job_title.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_summary = df2.job_summary.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:73: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_location = df2.job_location.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_quals = df2.job_quals.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_14060\1795372648.py:75: FutureWarning:

Starting
8
Number of jobs
25
/job/710140600
/job/710146000
/job/710194000
/job/710221700
/job/710227800
/job/710228400
/job/710265300
/job/710345000
/job/710356600
/job/710359200
/job/710363900
/job/710407100
/job/710441900
/job/710451500
/job/710453600
/job/710479400
/job/710521100
/job/710525100


In [ ]:
len(df)

In [ ]:
df_save = df